## Importa Biblioteca

In [1]:
import pandas as pd
import pandera as pa

#instalar bibliotecas em caso de erro comandos:
#pip install pandas
#pip install pandera

## Carrega arquivo

In [2]:
df = pd.read_csv("Arquivos_CENIPA/ocorrencia_2010_2020.csv", sep=';', parse_dates=['ocorrencia_dia'], dayfirst=True)
df #print
#df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0


In [19]:
df.loc[:, 'ocorrencia_cidade' : 'ocorrencia_uf'].value_counts()

ocorrencia_cidade  ocorrencia_uf
RIO DE JANEIRO     RJ               321
SÃO PAULO          SP               275
BELO HORIZONTE     MG               188
GUARULHOS          SP               153
CAMPINAS           SP               150
                                   ... 
NOVA MUTUM         MT                 1
NOVA ODESSA        SP                 1
COROATÁ            MA                 1
NOVA PONTE         MG                 1
PIACATU            SP                 1
Length: 1115, dtype: int64

In [20]:
df.describe()

,codigo_ocorrencia,codigo_ocorrencia2,total_recomendacoes
count,5752.000000,5752.000000,5752.000000
mean,58504.211926,58504.211926,0.303894
std,14060.291611,14060.291611,1.153253
min,39115.000000,39115.000000,0.000000
25%,46366.500000,46366.500000,0.000000
50%,52692.500000,52692.500000,0.000000
75%,77653.750000,77653.750000,0.000000
max,79874.000000,79874.000000,23.000000


## Cria Schema

In [4]:
schema = pa.DataFrameSchema(
    columns = {
        'codigo_ocorrencia': pa.Column(pa.Int),
        'codigo_ocorrencia2': pa.Column(pa.Int),
        'ocorrencia_classificacao': pa.Column(pa.String),
        'ocorrencia_cidade': pa.Column(pa.String),
        'ocorrencia_uf': pa.Column(pa.String, pa.Check.str_length(2,2)), ##str_length(tamanho_min, tamanho_max)
        'ocorrencia_aerodromo': pa.Column(pa.String),
        'ocorrencia_dia': pa.Column(pa.DateTime),
        'ocorrencia_hora': pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$')),
        'total_recomendacoes': pa.Column(pa.Int)
    }
)

## Trata campos invalidos

In [5]:
#df.dtypes  #verifica tipagem de campos
#df.isnull().sum() #verifica campos nullos

In [6]:
df.ocorrencia_hora.fillna('00:00:00', inplace= True)
df.ocorrencia_uf.replace('***', '  ', inplace= True)

#podemos permitir uma coluna com valores Null ao contruir o Schema 
#'ocorrencia_dia': pa.Column(pa.DateTime, nullable=True)

## Valida Schema

In [7]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,****,2020-12-31,09:00:00,0
